In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re

url="http://www.leagueofgraphs.com/champions/stats/diamond"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page.read(), "lxml")



In [2]:
for tr in soup.find_all('tr')[1:145]:
    tds = tr.find_all('td')
    
    if len(tds) >1:
        name = tds[1].text.split()
    
        champ_name = name[0].lower().replace("'", '').replace(",", '').replace(".", '')
        if champ_name == 'lee':
            champ_name = 'leesin'
        elif champ_name == 'miss':
            champ_name = 'missfortune'
        elif champ_name == 'twisted':
            champ_name = 'twistedfate'
        elif champ_name == 'master':
            champ_name = 'masteryi'
        elif champ_name == 'xin':
            champ_name = 'xinzhao'
        elif champ_name == 'dr':
            champ_name = 'drmundo'
        elif champ_name == 'tahm':
            champ_name = 'tahmkench'
        elif champ_name == 'aurelion':
            champ_name = 'aurelionsol'              
        elif champ_name == 'jarvan':
            champ_name = 'jarvaniv' 
            
        pop = float(tds[2].text.split()[0][0:-1])
        winrate = tds[3].text.split()[0][0:-1]
        banrate = tds[4].text.split()[0][0:-1]
        
#         print(champ_name, str(pop), str(winrate), str(banrate))
        
        with open('lolcurrent_raw_20170201.csv', 'a') as f:
            s = champ_name + ',' + str(pop) + ',' +  str(winrate) + ',' + \
                str(banrate) + '\n'
            f.write(s)

In [82]:
import pandas as pd
from datetime import datetime
import time

df = pd.read_csv('lolcurrent_raw_20170201.csv', names = ['name', 'popularity', 'winrate',
                                           'banrate'])
df_release = pd.read_csv('lolrelease.csv', names=['name', 'release'])

In [83]:
df = df.merge(df_release, how='outer', on='name')

In [84]:
df.dropna(inplace=True, axis=0)

In [85]:
df.head()

,name,popularity,winrate,banrate,release
0,jhin,37.5,51.9,0.7,1.451606e+09
1,leesin,35.7,48.4,8.6,1.270080e+09
2,ezreal,31.1,47.5,0.2,1.267402e+09
3,thresh,29.7,48.7,0.4,1.356998e+09
4,khazix,27.3,54.2,47.3,1.346458e+09


In [86]:

date = datetime.now()
date = time.mktime(date.timetuple())

df['tsr'] = df.apply(lambda x: date-x.release, axis=1)

In [87]:
df.head()

,name,popularity,winrate,banrate,release,tsr
0,jhin,37.5,51.9,0.7,1.451606e+09,34468975.0
1,leesin,35.7,48.4,8.6,1.270080e+09,215995375.0
2,ezreal,31.1,47.5,0.2,1.267402e+09,218673775.0
3,thresh,29.7,48.7,0.4,1.356998e+09,129076975.0
4,khazix,27.3,54.2,47.3,1.346458e+09,139617775.0


In [ ]:
# df_release = pd.read_csv('lolcurrent.csv', names=['name', 'release'])

In [13]:
# df.to_csv('lolcurrent.csv', index=False)

In [88]:
df_all = pd.read_csv('loldata_cleaned_v2.csv')
df_all.head()

,banrate,changes,date,freq_buff,freq_nerf,haschange,name,popularity,release,ts_buff,ts_nerf,tsr,vbr,vpop,vwr,winrate
0,0.13,nochange,1.395140e+09,5.0,5.0,False,aatrox,4.34,1.370045e+09,25.0,25.0,2.509560e+07,-0.31,-0.04,-2.02,49.72
1,0.19,nochange,1.395767e+09,21.0,16.0,False,aatrox,4.80,1.370045e+09,25.0,25.0,2.572200e+07,0.06,0.46,1.13,50.85
2,0.28,nochange,1.396411e+09,18.0,14.0,False,aatrox,5.06,1.370045e+09,25.0,25.0,2.636640e+07,0.09,0.26,0.03,50.88
3,0.20,nochange,1.397171e+09,14.0,18.0,False,aatrox,4.53,1.370045e+09,25.0,25.0,2.712601e+07,-0.08,-0.53,-1.40,49.48
4,0.34,nochange,1.397808e+09,14.0,16.0,False,aatrox,4.53,1.370045e+09,25.0,25.0,2.776320e+07,0.14,0.00,1.71,51.19


In [89]:
lastall = df_all[df_all['date']==df_all['date'].max()][['name', 'ts_buff', 'ts_nerf', 'popularity', 'winrate','banrate']]
lastall.head(10)

,name,ts_buff,ts_nerf,popularity,winrate,banrate
129,aatrox,25.0,25.0,0.82,46.38,0.77
259,ryze,6.0,24.0,11.67,45.78,18.69
388,sejuani,25.0,25.0,1.29,46.99,0.03
518,diana,25.0,24.0,2.17,46.85,0.17
648,shaco,25.0,25.0,9.04,52.63,4.02
778,zilean,25.0,25.0,4.92,52.65,0.15
908,shen,23.0,17.0,3.28,46.65,0.04
1038,shyvana,6.0,17.0,2.13,46.42,0.31
1167,darius,25.0,25.0,11.29,52.08,10.93
1296,singed,25.0,21.0,7.10,49.67,1.16


In [90]:
lastall['ts_buff'] = lastall['ts_buff'].apply(lambda x: x+1 if x <25 else 25)
lastall['ts_nerf'] = lastall['ts_nerf'].apply(lambda x: x+1 if x <25 else 25)
lastall.sort_values('name', inplace=True)
lastall.reset_index(inplace=True, drop=True)
lastall.head(10)

,name,ts_buff,ts_nerf,popularity,winrate,banrate
0,aatrox,25.0,25.0,0.82,46.38,0.77
1,ahri,25.0,25.0,14.56,52.30,1.17
2,akali,25.0,16.0,2.22,52.53,0.21
3,alistar,25.0,25.0,5.48,47.70,0.33
4,amumu,25.0,25.0,1.91,46.48,0.15
5,anivia,18.0,25.0,2.82,53.43,0.17
6,annie,25.0,25.0,4.08,50.37,0.32
7,ashe,24.0,25.0,9.26,50.01,0.07
8,aurelionsol,22.0,25.0,1.58,56.27,0.25
9,azir,25.0,16.0,1.75,45.98,0.14


In [91]:
df = df.merge(lastall[['name', 'ts_buff', 'ts_nerf']], on='name')
df.sort_values('name', inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,name,popularity,winrate,banrate,release,tsr,ts_buff,ts_nerf
0,aatrox,0.6,45.8,0.7,1.370045e+09,116030575.0,25.0,25.0
1,ahri,13.9,52.3,1.0,1.322698e+09,163377775.0,25.0,25.0
2,akali,2.6,50.5,0.3,1.272672e+09,213403375.0,25.0,16.0
3,alistar,4.5,48.9,0.2,1.233446e+09,252628975.0,25.0,25.0
4,amumu,1.5,44.5,0.1,1.243814e+09,242260975.0,25.0,25.0


In [92]:
df['vpop'] = df['popularity']-lastall['popularity']
df['vwr'] = df['winrate']-lastall['winrate']
df['vbr'] = df['banrate']-lastall['banrate']

In [93]:
df.head()

,name,popularity,winrate,banrate,release,tsr,ts_buff,ts_nerf,vpop,vwr,vbr
0,aatrox,0.6,45.8,0.7,1.370045e+09,116030575.0,25.0,25.0,-0.22,-0.58,-0.07
1,ahri,13.9,52.3,1.0,1.322698e+09,163377775.0,25.0,25.0,-0.66,0.00,-0.17
2,akali,2.6,50.5,0.3,1.272672e+09,213403375.0,25.0,16.0,0.38,-2.03,0.09
3,alistar,4.5,48.9,0.2,1.233446e+09,252628975.0,25.0,25.0,-0.98,1.20,-0.13
4,amumu,1.5,44.5,0.1,1.243814e+09,242260975.0,25.0,25.0,-0.41,-1.98,-0.05


In [94]:
# df.to_csv('lolcurrent.csv', index=False)

In [83]:
from sklearn.externals import joblib
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import pandas as pd

df = pd.read_csv('lolcurrent.csv')
xgbc = joblib.load('xgbc.dat') 
p_xgb_o = xgbc.predict(df[['popularity', 'winrate', 'banrate',
                            'tsr', 'ts_buff', 'ts_nerf', 'vpop', 'vbr', 'vwr']].as_matrix())
prob_xgb_o = xgbc.predict_proba(df[['popularity', 'winrate', 'banrate', 'tsr', 'ts_buff', 'ts_nerf', 'vpop','vbr', 'vwr']].as_matrix())
dfp = pd.DataFrame(p_xgb_o, columns=['changes'])

dfn = pd.DataFrame(df.iloc[dfp[dfp['changes'] == 'nerf'].index]['name'])
nerfs_html = dfn.transpose().to_html(index=False)

dfb = pd.DataFrame(df.iloc[dfp[dfp['changes'] == 'buff'].index]['name'])
buffs_html = dfb.transpose().to_html(index=False)




In [139]:
prob_xgb_o[df.iloc[dfp[dfp['changes'] == 'buff'].index].index]
# df.iloc[dfp[dfp['changes'] == 'nerf'].index]

array([[ 0.47605851,  0.10186732,  0.42207423],
       [ 0.48798439,  0.05163356,  0.46038201],
       [ 0.54276407,  0.05496356,  0.40227237],
       [ 0.57606673,  0.31309062,  0.11084259]], dtype=float32)

In [123]:
df_nerf_probs['dates'] = pd.to_datetime(df_nerf_probs['dates'],unit='s').dt.date

,camille,jayce,khazix,leblanc,malzahar,rengar,dates
0,NaN,0.066998,0.344901,0.269095,0.365335,0.203174,2016-06-25
1,NaN,0.143312,0.561865,0.248455,0.709333,0.302725,2016-07-05
2,NaN,0.083045,0.449456,0.313123,0.966722,0.217538,2016-07-13
3,NaN,0.120772,0.421260,0.267658,0.968867,0.278898,2016-07-20
4,NaN,0.134851,0.398185,0.395909,0.735802,0.300143,2016-07-30


In [125]:

# Format probabilities for predicted nerfs
df_nerf_probs = pd.read_csv('lolcurrent_nerf_probs.csv')
champs_nerf = list(dfn['name'].values)
champs_nerf.append('dates')
df_nerf_probs = df_nerf_probs[champs_nerf]
df_nerf_probs['dates'] = pd.to_datetime(df_nerf_probs['dates'],unit='s').dt.date
df_nerf_probs.to_csv('graph_nerfs.csv', index=False, na_rep='NaN')

# Format probabilities for predicted buffs
df_buff_probs = pd.read_csv('lolcurrent_buff_probs.csv')
champs_buff = list(dfb['name'].values)
champs_buff.append('dates')
df_buff_probs = df_buff_probs[champs_buff]
df_buff_probs['dates'] = pd.to_datetime(df_buff_probs['dates'],unit='s').dt.date
df_buff_probs.to_csv('graph_buffs.csv', index=False, na_rep='NaN')




# plt.figure(figsize=(20,10))
           
# for champ in df.iloc[dfp[dfp['changes']=='nerf'].index]['name']:
# #     champ = df.iloc[dfp[dfp['changes']=='nerf'].index]['name'].iloc[0]
#     data = df_probs[df_probs['name']==champ]['nerf']
#     if len(data) > 15:
#         ax = sns.tsplot(data=data, color = 'red')
        
# plt.legend()




In [61]:
data

,dates,lulu,skarner,varus,twitch,blitzcrank,annie,katarina,veigar,udyr,...,darius,taliyah,kogmaw,trundle,tristana,shen,kennen,missfortune,ahri,soraka
0,1.466892e+09,0.085693,0.207152,0.312601,0.085677,0.783719,0.110994,0.201665,0.156975,0.296434,...,0.096875,0.170004,0.158110,0.941265,0.340586,0.128797,0.128596,0.156475,0.127383,0.034903
1,1.467760e+09,0.045064,0.169600,0.372725,0.070176,0.873710,0.106621,0.180591,0.247621,0.489874,...,0.081873,0.073545,0.173085,0.930249,0.196096,0.236397,0.243280,0.173989,0.101121,0.039593
2,1.468397e+09,0.193643,0.152703,0.318184,0.066010,0.146062,0.920253,0.235485,0.301660,0.428864,...,0.077573,0.065888,0.181564,0.335608,0.258240,0.209073,0.159978,0.228605,0.112351,0.063484
3,1.469041e+09,0.262255,0.144049,0.285787,0.072398,0.186923,0.144403,0.172629,0.284802,0.317653,...,0.118604,0.040074,0.191172,0.302272,0.224522,0.041090,0.137863,0.196822,0.106508,0.041606
4,1.469840e+09,0.338039,0.184986,0.372725,0.077141,0.158728,0.181503,0.146645,0.306366,0.307058,...,0.097830,0.045012,0.142916,0.402761,0.135816,0.040953,0.114512,0.239130,0.109142,0.045012
5,1.470614e+09,0.944058,0.134613,0.401218,0.064418,0.122761,0.097974,0.125209,0.193880,0.261505,...,0.094244,0.081770,0.094060,0.423504,0.099352,0.166183,0.162504,0.169827,0.081060,0.051343
6,1.471493e+09,0.855307,0.149096,0.404628,0.065205,0.137741,0.104248,0.217060,0.280694,0.339201,...,0.067799,0.090844,0.106927,0.310401,0.125632,0.161841,0.113880,0.221984,0.096816,0.048422
7,1.472663e+09,0.445230,0.115562,0.408594,0.050673,0.122761,0.099688,0.170571,0.859760,0.373944,...,0.099883,0.090278,0.092318,0.919362,0.160512,0.156875,0.879648,0.211797,0.099546,0.047398
8,1.473595e+09,0.852706,0.105035,0.292643,0.065070,0.150711,0.102302,0.161722,0.172819,0.443254,...,0.053223,0.626155,0.063178,0.348124,0.097002,0.069508,0.156572,0.837192,0.118403,0.041428
9,1.474405e+09,0.295205,0.153586,0.327041,0.028555,0.075804,0.099399,0.097891,0.194410,0.950602,...,0.100364,0.309707,0.072563,0.370361,0.135620,0.176694,0.203393,0.176442,0.102163,0.011565


In [72]:
group = df_probs.groupby('name')

data = pd.DataFrame(df_probs['dates'].unique(), columns=['dates'])

for champ in list(group.groups.keys()):

    df_temp = group.get_group(champ)
    df_temp = df_temp[['dates', 'nochange']]
    df_temp.columns = ['dates', champ]

    data = pd.merge(data, df_temp, on='dates', how='outer')

data = data.iloc[0:30]
data

,dates,lulu,skarner,varus,twitch,blitzcrank,annie,katarina,veigar,udyr,...,darius,taliyah,kogmaw,trundle,tristana,shen,kennen,missfortune,ahri,soraka
0,1.466892e+09,0.044615,0.701507,0.589717,0.767733,0.090374,0.818453,0.726222,0.746689,0.636988,...,0.511924,0.560071,0.632616,0.032444,0.604301,0.645290,0.802328,0.752044,0.679792,0.673135
1,1.467760e+09,0.034326,0.718586,0.578024,0.733464,0.041779,0.800114,0.728885,0.694169,0.457063,...,0.473654,0.313422,0.680829,0.033056,0.733708,0.410801,0.666861,0.757740,0.745876,0.308642
2,1.468397e+09,0.649747,0.756113,0.551066,0.734794,0.716025,0.062953,0.668399,0.638191,0.515437,...,0.508542,0.033361,0.709428,0.591012,0.695464,0.473412,0.733874,0.698723,0.703489,0.568622
3,1.469041e+09,0.623172,0.733097,0.644184,0.705728,0.711786,0.806954,0.722821,0.660890,0.561645,...,0.591919,0.045762,0.745197,0.619692,0.674805,0.036196,0.723409,0.739172,0.712473,0.615613
4,1.469840e+09,0.551083,0.669115,0.578024,0.628240,0.662584,0.741621,0.737381,0.589754,0.630761,...,0.485596,0.059424,0.400588,0.500893,0.806190,0.036853,0.574745,0.634394,0.710158,0.401665
5,1.470614e+09,0.036011,0.758547,0.559306,0.635355,0.771966,0.780055,0.683026,0.699301,0.652694,...,0.574738,0.700889,0.432893,0.513769,0.774180,0.673086,0.644321,0.765501,0.716762,0.479116
6,1.471493e+09,0.096461,0.730955,0.504722,0.732594,0.731389,0.813262,0.614552,0.647584,0.603930,...,0.644748,0.319513,0.558395,0.600186,0.776200,0.671439,0.702383,0.698982,0.757756,0.451856
7,1.472663e+09,0.498378,0.551594,0.509670,0.357301,0.771966,0.735576,0.693377,0.084928,0.580216,...,0.541230,0.387851,0.370021,0.052123,0.725349,0.521002,0.058240,0.673158,0.721412,0.307910
8,1.473595e+09,0.089782,0.779216,0.638018,0.687439,0.689982,0.783064,0.761075,0.777545,0.504736,...,0.533193,0.109147,0.229347,0.598220,0.757369,0.064397,0.588157,0.103120,0.588666,0.317309
9,1.474405e+09,0.624639,0.674340,0.592339,0.106310,0.629386,0.800741,0.821038,0.766409,0.029062,...,0.579226,0.601600,0.660544,0.578557,0.770789,0.695350,0.476281,0.673443,0.697757,0.032324


In [73]:
# data.to_csv('lolcurrent_buff_probs.csv', index=False)
# data.to_csv('lolcurrent_nerf_probs.csv', index=False)
# data.to_csv('lolcurrent_nochange_probs.csv', index=False)

In [ ]:
sudo docker run --rm -d --name flask1 -p 80:80 josephlee021/flaskimage:version004

In [ ]:
sudo docker exec -i -t flask1 /bin/bash

In [ ]:
sudo docker stop flask1
sudo docker build -t joe/flaskimage:version004 .

In [ ]:
sudo ssh -i insight.pem ec2-user@ec2-52-10-107-25.us-west-2.compute.amazonaws.com

# blue trinket.gg